In [1]:
#https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

Get file names


In [2]:
# File and folder paths
dirpath = r"../Data/GEE/"

# Make a search criteria to select the DEM files
search_criteria = "SL_mos*.tif"

q = os.path.join(dirpath, search_criteria)

print(q)


GEE = glob.glob(q)
GEE

../Data/GEE/SL_mos*.tif


['../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000000000.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000010496.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000020992.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000031488.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000041984.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000052480.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000062976.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000073472.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000000000-0000083968.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000010496-0000000000.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000010496-0000010496.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000010496-0000020992.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000010496-0000031488.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI_Stats-0000010496-0000041984.tif',
 '../Data/GEE\\SL_mosaic_2017_NDVI

Empty list for the datafiles that will be part of the mosaic. 

open all those files in read mode with raterio and add those files into a our source file list

In [3]:
src_files_to_mosaic = []

for fp in GEE:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)
 

In [4]:
list(src_files_to_mosaic[0].indexes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [5]:
# Copy the metadata
out_meta = src.meta.copy()

out_meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 9836,
 'height': 1966,
 'count': 10,
 'crs': CRS({'init': 'epsg:4326'}),
 'transform': Affine(8.983152841195215e-05, 0.0, -85.2254167926293,
        0.0, -8.983152841195215e-05, 16.810802732427486)}

Merge those together band by band and create a mosaic with rasterio’s merge function.

In [ ]:

# iterate through bands and write to file 
for band in list(src_files_to_mosaic[0].indexes):
    print('working on band'+str(band))
    
    # mosaic tiles 
    mosaic, out_trans = merge(src_files_to_mosaic,indexes=band)
    
    # Update the metadata
    out_meta.update({"driver": "GTiff",
                  "height": mosaic.shape[1],
                  "width": mosaic.shape[2],
                  'count': 1,
                  "transform": out_trans 
                  }  )
    
    # set destination
    out_fp = r"../Data/GEE/Sentinel_Mosaic_B"+str(band)+".tif"
    
    # write 
    with rasterio.open(out_fp, "w", **out_meta) as dest:
         dest.write(mosaic)

working on band1


In [ ]:
%matplotlib inline
show(mosaic, cmap='terrain')